In [142]:
import re
import numpy as np
from tqdm import tqdm
import mmap
import time

In [3]:
question_path='data/question.txt'
sentence_path='data/sentence.txt'
embs_path='data/embs.npy'

In [72]:
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    fp.close()
    return lines

def parse_words(filen_name):
    with open(filen_name,"r") as file:
        text = re.sub(r'\W+', ' ', file.read())
        text = re.sub(r' +',' ',text)
        return set(text.split())

def create_words_dict():
    q_words=parse_words(question_path)
    s_words=parse_words(sentence_path)
    words=q_words.union(s_words)
    id_2_word=dict(enumerate(words))
    return id_2_word,dict(zip(id_2_word.values(), id_2_word.keys()))

def str_2_id(x):
    return word_2_id[x]

def prepare_data(text):
    X=[]
    with open(text,'r') as f:
        for line in tqdm(f,total=get_num_lines(text)):
            line = re.sub(r'\W+', ' ', line)
            line = re.sub(r' +',' ',line)
            X.append(np.array(list(map(str_2_id,line.split()))+[eos_token]))
        
    
    return np.array(X)

In [99]:
id_2_word,word_2_id=create_words_dict()
embeddings=np.load(embs_path)
embeddings=np.vstack([embeddings,np.zeros([1,300])])
embeddings=np.vstack([embeddings,np.zeros([1,300])])
print('> Vocab Len : ',len(id_2_word)+2)
eos_token=len(id_2_word)
sos_token=len(id_2_word)+1
vocab_size=len(id_2_word)+2
print('> Embs Shape : ',embeddings.shape)

> Vocab Len :  64940
> Embs Shape :  (64940, 300)


In [100]:
print('> Loading Questions ')
Qs=prepare_data(question_path)
print('> Loading Sentences ')
As=prepare_data(sentence_path)

  3%|▎         | 2218/70484 [00:00<00:03, 22174.21it/s]

> Loading Questions 


  6%|▌         | 4282/70484 [00:00<00:03, 21402.90it/s]

> Loading Sentences 


100%|██████████| 70484/70484 [00:03<00:00, 21699.33it/s]


In [8]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [208]:
class Encoder(nn.Module):
    def __init__(self,input_size,hidden_size,embs):
        super(Encoder,self).__init__()
        self.hidden_size=hidden_size
        self.embedding=nn.Embedding(input_size,hidden_size)
        self.embedding.weight.data.copy_(torch.from_numpy(embs).cuda())
        self.gru=nn.GRU(hidden_size,hidden_size)
    
    def forward(self,inputt,hidden):
        embedded=self.embedding(inputt).view(1,1,-1)
        output=embedded
        output,hidden=self.gru(output,hidden)
        return output,hidden

    def init_hidden(self):
        result=Variable(torch.zeros(1,1,self.hidden_size))
        return result.cuda()
    
    
    

In [209]:
# test Encoder
input_size=vocab_size
hidden_size=300

enc=Encoder(input_size,hidden_size,embeddings)
enc_hidden=enc.init_hidden()



In [210]:
sentence=Variable(torch.from_numpy(As[0])).view(-1,1).cuda()
print(sentence[0].size())
enc=enc.cuda()
# print(sentence[:,0])
# # print(sentence[0].view(1,1,-1))
for i in range(As[0].shape[0]):
    _,enc_hidden=enc.forward(sentence[i],enc_hidden)

print(enc_output.shape)



torch.Size([1])
torch.Size([1, 1, 300])


In [211]:
class Decoder(nn.Module):
    def __init__(self,hidden_size,output_size,embs):
        super(Decoder,self).__init__()
        self.hidden_size=hidden_size
        self.embedding=nn.Embedding(input_size,hidden_size)
        self.embedding.weight.data.copy_(torch.from_numpy(embs).cuda())
        
        self.gru=nn.GRU(hidden_size,hidden_size)
        self.out=nn.Linear(hidden_size,output_size)
        self.softmax=nn.LogSoftmax(dim=1)
    
    def forward(self,inputt,hidden):
        output=self.embedding(inputt).view(1,1,-1)
        output=F.relu(output)
        output,hidden=self.gru(output,hidden)
        output=self.out(output[0])
        output=self.softmax(output)
        return output,hidden    

In [198]:
dec=Decoder(hidden_size,input_size,embeddings)

<class 'torch.nn.parameter.Parameter'>


In [134]:
sos=Variable(torch.LongTensor([input_size-1])).view(-1,1)
print(sos[0].size())
dec.embedding(sos[0]).view(1,-1,1)
o,h=dec.forward(sos[0],enc_hidden)
print(o.shape)

torch.Size([1])
torch.Size([1, 64940])


In [30]:
x = np.array([1, 2, 25])
x=x.astype('float32')
x

array([ 1.,  2., 25.], dtype=float32)

In [217]:
def train(sentence,question,encoder,decoder,enc_opt,dec_opt,criterion,max_length=100):
    t=time.time()
    encoder_hidden=encoder.init_hidden()
    
    enc_opt.zero_grad()
    dec_opt.zero_grad()
    
    sentence=Variable(torch.from_numpy(sentence)).view(-1,1).cuda()
    question=Variable(torch.from_numpy(question)).view(-1,1).cuda()
    
    s_len=sentence.size()[0]
    q_len=question.size()[0]
    
    loss=0
    for ei in range(s_len):
        encoder_output,encoder_hidden=encoder.forward(sentence[ei],encoder_hidden)
    
    te=time.time()
#     print('encoder forward',te-t)
    sos=Variable(torch.LongTensor([input_size-1])).view(-1,1).cuda()
    decoder_hidden=encoder_hidden
    
    for di in range(q_len):
        decoder_output,decoder_hidden=decoder(sos,decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        loss+=criterion(decoder_output,question[di])
    
        if ni==eos_token:
            break
    
    td=time.time()
#     print('decoder forward',td-te)
    
    loss.backward()
    enc_opt.step()
    dec_opt.step()
#     print('backward',time.time()-td)
    return loss.data[0]/q_len

In [218]:
input_size=vocab_size
hidden_size=300

enc=Encoder(input_size,hidden_size,embeddings).cuda()
dec=Decoder(hidden_size,input_size,embeddings).cuda()

enc_opt=optim.SGD(enc.parameters(),lr=0.01)
dec_opt=optim.SGD(dec.parameters(),lr=0.01)

criterion=nn.NLLLoss()

print_every=100
loss=0
for i in tqdm(range(As.shape[0])):
    loss += train(As[i],Qs[i],enc,dec,enc_opt,dec_opt,criterion)
    if(i%print_every==0):
        print('Loss : ',loss/print_every)
        loss=0

  0%|          | 2/70484 [00:00<1:49:46, 10.70it/s]

Loss :  0.11086773872375488


  0%|          | 103/70484 [00:10<1:55:30, 10.16it/s]

Loss :  8.44414954750905


  0%|          | 203/70484 [00:20<1:55:35, 10.13it/s]

...

Loss :  4.551400270717946


  2%|▏         | 1402/70484 [03:05<2:32:29,  7.55it/s]

Loss :  4.470440572433788


  2%|▏         | 1432/70484 [03:11<2:33:57,  7.48it/s]

KeyboardInterrupt: 

In [196]:
type(torch.rand([3,2]).cuda())

torch.cuda.FloatTensor